In [49]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
# from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

In [3]:
model_name = "Qwen/Qwen1.5-1.8B-Chat"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500, temperature=0.6, do_sample=True)

llm = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cpu


In [6]:
llm.invoke("who are you")

"who are you? i am a large language model created by Alibaba Cloud. my name is QianWen, and I am designed to assist with various tasks and answer questions to the best of my ability. I can understand natural language inputs in multiple languages, including English, Chinese, Spanish, French, German, Italian, Japanese, Korean, and more.\n\nQianWen uses advanced deep learning algorithms and neural networks to process and generate human-like text based on the input provided. I have been trained on a vast corpus of text data, including books, articles, websites, and other sources, which allows me to provide accurate and relevant information on a wide range of topics.\n\nWhether you need assistance with homework, want to learn new things, or simply want to communicate with someone in another language, QianWen has got you covered. Whether you're looking for information about a specific topic, asking a question, or engaging in a conversation, I'm here to help.\n\nOverall, QianWen is a powerful

In [9]:
loader = WebBaseLoader("https://www.geeksforgeeks.org/introduction-to-langchain/")
docs = loader.load()

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [15]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [16]:
len(documents)

6

In [ ]:
print(documents[0].page_content)

In [20]:
vector_store = FAISS.from_documents(documents, embeddings)

In [21]:
print(vector_store)

In [50]:
template = """" 
Answer the following question based on the given context only:
<context>
{context}
</context>

Question: {input}
"""


prompt = PromptTemplate.from_template(template)
new_prompt = ChatPromptTemplate.from_template(template)

In [51]:
document_chain = create_stuff_documents_chain(llm=llm, prompt=new_prompt)

In [52]:
document_chain.invoke({
    "input": "What is your name?",
    "context": [Document(page_content="I'm john, currently learning langchain")]
})

'Human: " \nAnswer the following question based on the given context only:\n<context>\nI\'m john, currently learning langchain\n</context>\n\nQuestion: What is your name?\nAnswer: John.'

# Reteival Chain

In [53]:
retriver = vector_store.as_retriever()
retrival_chain = create_retrieval_chain(retriver, document_chain)

In [54]:
response = retrival_chain.invoke({
    "input": "Give me summary of about 50 words about this document"
})

In [55]:
response

{'input': 'Give me summary of about 50 words about this document',
 'context': [Document(id='750696c7-c4c0-4ea1-ae73-d57a716d39a5', metadata={'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="matches the user's query.3. Fetching Relevant InformationBased on the similarity search, LangChain retrieves the most relevant data or context from the database. This step ensures that the language model has access to accurate and contextually appropriate information to generate a meaningful response.4. Generating a ResponseThe retrieved information is passed to the\xa0language model\xa0(e.g., OpenAI's GPT, Anthropic's Clau

In [56]:
response["answer"]

'Human: " \nAnswer the following question based on the given context only:\n<context>\nmatches the user\'s query.3. Fetching Relevant InformationBased on the similarity search, LangChain retrieves the most relevant data or context from the database. This step ensures that the language model has access to accurate and contextually appropriate information to generate a meaningful response.4. Generating a ResponseThe retrieved information is passed to the\xa0language model\xa0(e.g., OpenAI\'s GPT, Anthropic\'s Claude, or others). The LLM processes the input and generates a response or takes an action based on the provided data. For example, if the query is about the weather, the LLM might generate a response like,\xa0“Today’s weather is sunny with a high of 75°F.”The formatted response is returned to the user as the final output. The user receives a clear, accurate, and contextually relevant answer to their query.Getting Started with LangChainTo get started with LangChain, you\'ll need to

## Conversational Retrieval Chain

In [57]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Gven the above conversation, generate a search query to look up inorder to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriver, prompt)

In [58]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="What is the latest version og langchain?"),
    AIMessage(content="Langchain 0.1.0")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it."
})

[Document(id='750696c7-c4c0-4ea1-ae73-d57a716d39a5', metadata={'source': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'title': 'Introduction to LangChain - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="matches the user's query.3. Fetching Relevant InformationBased on the similarity search, LangChain retrieves the most relevant data or context from the database. This step ensures that the language model has access to accurate and contextually appropriate information to generate a meaningful response.4. Generating a ResponseThe retrieved information is passed to the\xa0language model\xa0(e.g., OpenAI's GPT, Anthropic's Claude, or others). The LLM processes the input and generates a response or takes a